In [3]:
import casadi as ca
import numpy as np
import math

In [4]:
roll = ca.SX.sym('roll')
pitch = ca.SX.sym('pitch')
yaw = ca.SX.sym('yaw')

r_roll = ca.SX_eye(3)
r_roll[1:,1:] = np.array([[ca.cos(roll), -ca.sin(roll)],
                 [ca.sin(roll), ca.cos(roll)]])
r_pitch = ca.SX_eye(3)
r_pitch[[0,2],[0,2]] = np.array([[ca.cos(pitch), ca.sin(pitch)],
                 [-ca.sin(pitch), ca.cos(pitch)]])
r_yaw = ca.SX_eye(3)
r_yaw[:2,:2] = np.array([[ca.cos(yaw), -ca.sin(yaw)],
                 [ca.sin(yaw), ca.cos(yaw)]])

print("ROLL", r_roll)
print("PITCH", r_pitch)
print("YAW", r_yaw)

R = (r_yaw @ r_pitch @ r_roll)
print("Y-P-R")
for i in range(0,3):
    for j in range(0,3):
        print(R[i, j],end=' , ')
    print()

print(ca.Function('rot', [yaw, pitch], [r_yaw @ r_pitch])(11*math.pi/180, 68*math.pi/180))


ROLL 
[[1, 00, 00], 
 [00, cos(roll), (-sin(roll))], 
 [00, sin(roll), cos(roll)]]
PITCH 
[[cos(pitch), 00, sin(pitch)], 
 [00, 1, 00], 
 [(-sin(pitch)), 00, cos(pitch)]]
YAW 
[[cos(yaw), (-sin(yaw)), 00], 
 [sin(yaw), cos(yaw), 00], 
 [00, 00, 1]]
Y-P-R
(cos(yaw)*cos(pitch)) , (((cos(yaw)*sin(pitch))*sin(roll))-(sin(yaw)*cos(roll))) , ((sin(yaw)*sin(roll))+((cos(yaw)*sin(pitch))*cos(roll))) , 
(sin(yaw)*cos(pitch)) , ((cos(yaw)*cos(roll))+((sin(yaw)*sin(pitch))*sin(roll))) , (((sin(yaw)*sin(pitch))*cos(roll))-(cos(yaw)*sin(roll))) , 
(-sin(pitch)) , (cos(pitch)*sin(roll)) , (cos(pitch)*cos(roll)) , 

[[0.367724, -0.190809, 0.910149], 
 [0.0714783, 0.981627, 0.176915], 
 [-0.927184, 00, 0.374607]]


In [5]:
## For magnetometer measurement correction
print("Correction : Intrinsic P-R")
R = r_pitch @ r_roll
for i in range(0,3):
    for j in range(0,3):
        print(R[i, j],end=' , ')
    print()

Correction : Intrinsic P-R
cos(pitch) , (sin(pitch)*sin(roll)) , (sin(pitch)*cos(roll)) , 
00 , cos(roll) , (-sin(roll)) , 
(-sin(pitch)) , (cos(pitch)*sin(roll)) , (cos(pitch)*cos(roll)) , 


## Complementary Filter Derivation
For our complementary filter we use an intrinsic Y-P-R sequence, meaning from the inertial frame we first rotate by a yaw angle $\psi$ then by a pitch angles $\theta$ then by a roll angle $\phi$ giving our rotation from inertial to body as $${}^{I}R_{B}=R_{\psi}R_{\theta}R_{\phi}$$
$$R=\begin{bmatrix}
\cos{\psi} & -\sin{\psi} & 0 \\
\sin{\psi} & \cos{\psi} & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
\cos{\theta} & 0 & \sin{\theta} \\
0 & 1 & 0 \\
-\sin{\theta} & 0 & \cos{\theta} \\
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos{\phi} & -\sin{\phi} \\
0 & \sin{\phi} & \cos{\phi}
\end{bmatrix}$$

This rotation can translate a vector expressed in the body frame to the inertial frame via:
$$\textbf{v}_{I}={}^{I}R_{B}\textbf{v}_{B}$$

Likewise, we can also transform a vector expressed in the inertial frame to the body frame via:
$$\textbf{v}_{B}=({}^{I}R_{B})^{T}\textbf{v}_{I}$$

We have ${}^{I}R_{B}=R_{\psi}R_{\theta}R_{\phi}$ which we can expand to $({}^{I}R_{B})^{T}=R^{T}_{\phi}R^{T}_{\theta}R^{T}_{\psi}$ noting the change in the order.

### Accelerometer Derivation - Roll and Pitch
While our accelerometer measurements are in the body-frame, we know the gravitational vector in the (NED) inertial frame is
$$\textbf{a}_{I}=\begin{bmatrix}0 \\ 0 \\ -g\end{bmatrix}$$

We will solve the following equation to obtain our tilt angles $\phi$ and $\theta$
$$\begin{bmatrix}a_{x} \\ a_{y} \\ a_{z}\end{bmatrix}=({}^{I}R_{B})^{T}\begin{bmatrix}0 \\ 0 \\ -g\end{bmatrix}$$

Assuming no yaw ($\psi=0$), we know our tilt is defined as:
$$\begin{bmatrix}
a_{x} \\
a_{y} \\
a_{z} \\
\end{bmatrix}=
\begin{bmatrix}
\cos{\theta} & 0 & -\sin{\theta} \\
\sin{\phi}\sin{\theta} & \cos{\phi} & \sin{\phi}\cos{\theta} \\
\cos{\phi}\sin{\theta} & -\sin{\phi} & \cos{\phi}\cos{\theta} \\
\end{bmatrix}
\begin{bmatrix}
0 \\
0 \\
-g \\
\end{bmatrix}
$$

Expanding gives us these equations$$
\begin{align}
g\sin{\theta}&=a_{x} \\
-g\sin{\phi}\cos{\theta}&=a_{y} \\
-g\cos{\phi}\cos{\theta}&=a_{z}
\end{align}$$

We can divide the second row by the third row to get
$$\tan{\phi} = \frac{a_{y}}{a_{z}}$$
Because we are in the NED frame, $a_{z}$ will be negative between $\frac{\pi}{2}\leq\phi\leq\frac{\pi}{2}$ which means we need to negate $a_{z}$. We also then negate $a_{y}$ to keep the equation consistent.
$$\phi=atan2(-a_{y},-a_{z}), -\frac{\pi}{2}\leq\phi\leq\frac{\pi}{2}$$

For the pitch, we can square both sides of the second and third equations to get
$$g^{2}\sin^{2}{\phi}\cos^{2}{\theta}=a_{y}^{2}$$
$$g^{2}\cos^{2}{\phi}\cos^{2}{\theta}=a_{z}^{2}$$

We also square the first equation to get
$$g^{2}=\frac{a_{x}}{\sin{\theta}}$$

We can sum the first two squared equations, and then subsitute g from the other equation as follows
$$g^{2}\sin^{2}{\phi}\cos^{2}{\theta} + g^{2}\cos^{2}{\phi}\cos^{2}{\theta}=a_{z}^{2}+a_{y}^{2}$$
$$g^{2}\cos^{2}{\theta}=a_{z}^{2}+a_{y}^{2}$$
$$\frac{a_{x}^{2}\cos^{2}{\theta}}{\sin^{2}{\theta}}=a_{z}^{2}+a_{y}^{2}$$

Re-arranging and taking the square root of both sides gives us
$$\tan{\theta}=\frac{a_{x}}{\sqrt{a_{z}^{2}+a_{y}^{2}}}$$
$$\theta=\text{atan2}(a_{x},\sqrt{a_{z}^{2}+a_{y}^{2}}),-\pi\leq\theta\leq\pi$$

### Magnetometer Derivation
Again, we can re-use the roll/pitch rotation to transform our body-frame magnetometer reading into a 'yaw-only' frame with x and y axes aligned to the NED frame X-Y plane
$$\begin{align}
\begin{bmatrix}
m_{x,comp} \\
m_{y,comp} \\
m_{z,comp} \\
\end{bmatrix}&=
\begin{bmatrix}
\cos{\theta} & \sin{\theta}\sin{\phi} & \sin{\theta}\cos{\phi} \\
0 & \cos{\phi} & -\sin{\phi} \\
-\sin{\theta} & \cos{\theta}\sin{\phi} & \cos{\theta}\cos{\phi} \\
\end{bmatrix}
\begin{bmatrix}
m_{x} \\
m_{y} \\
m_{z} \\
\end{bmatrix} \\
&=\begin{bmatrix}
m_{x}\cos{\theta} + m_{y}\sin{\theta}\sin{\phi} + m_{z}\sin{\theta}\cos{\phi} \\
m_{y}\cos{\phi} -m_{z}\sin{\phi} \\
-m_{x}\sin{\theta} + m_{y}\cos{\theta}\sin{\phi} + m_{z}\cos{\theta}\cos{\phi} \\
\end{bmatrix}
\end{align}
$$
In our 'yaw-only' frame, and ignoring magnetic declination for now, we can relate the magnetic field vector to our (tilt-compensated) reading via
$$
B_{ref}=
\begin{bmatrix}
b_{x} \\
0 \\
b_{z}
\end{bmatrix}
= \begin{bmatrix}
\cos{\psi} & -\sin{\psi} & 0 \\
\sin{\psi} & \cos{\psi} & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
m_{x,comp} \\
m_{y,comp} \\
m_{z,comp} \\
\end{bmatrix}
$$
Now our $B_{ref}$ vector has x and z components only and hence we can use the second equation to compute yaw
$$\psi=\text{atan2}(-m_{y,comp},m_{x,comp})$$

The last step is to offset this yaw measurement by the magnetic declination for the current location.

In [10]:
roll = ca.SX.sym('roll')
pitch = ca.SX.sym('pitch')
yaw = ca.SX.sym('yaw')

r_roll = ca.SX_eye(3)
r_roll[1:,1:] = np.array([[ca.cos(roll), -ca.sin(roll)],
                 [ca.sin(roll), ca.cos(roll)]])
r_pitch = ca.SX_eye(3)
r_pitch[[0,2],[0,2]] = np.array([[ca.cos(pitch), ca.sin(pitch)],
                 [-ca.sin(pitch), ca.cos(pitch)]])
r_yaw = ca.SX_eye(3)
r_yaw[:2,:2] = np.array([[ca.cos(yaw), -ca.sin(yaw)],
                 [ca.sin(yaw), ca.cos(yaw)]])

print("ROLL", r_roll)
print("PITCH", r_pitch)
print("YAW", r_yaw)

print(r_pitch @ r_roll)
print('--')
for i in range(0,3):
    for j in range(0,3):
        print((r_roll.T @ r_pitch.T)[i, j], end=' , ')
    print()

ROLL 
[[1, 00, 00], 
 [00, cos(roll), (-sin(roll))], 
 [00, sin(roll), cos(roll)]]
PITCH 
[[cos(pitch), 00, sin(pitch)], 
 [00, 1, 00], 
 [(-sin(pitch)), 00, cos(pitch)]]
YAW 
[[cos(yaw), (-sin(yaw)), 00], 
 [sin(yaw), cos(yaw), 00], 
 [00, 00, 1]]
@1=sin(pitch), @2=sin(roll), @3=cos(pitch), @4=cos(roll), 
[[cos(pitch), (@1*@2), (@1*@4)], 
 [00, cos(roll), (-sin(roll))], 
 [(-sin(pitch)), (@3*@2), (@3*@4)]]
--
cos(pitch) , 00 , (-sin(pitch)) , 
(sin(roll)*sin(pitch)) , cos(roll) , (sin(roll)*cos(pitch)) , 
(cos(roll)*sin(pitch)) , (-sin(roll)) , (cos(roll)*cos(pitch)) , 


## Euler Rates from Gyroscope
Our gyroscope measures angular velocity in the body-fixed frame however, our state vector uses euler angles (and their derivatives, the euler rates) in sequentially rotated frames, so we need to transform each back. Our roll rate is actually in the same frame as the gyroscope (body-fixed frame) but the other two require rotations. The full equation:
$$\begin{align}\begin{bmatrix}
p \\ q \\ r
\end{bmatrix}
&=\begin{bmatrix} \dot{\phi} \\ 0 \\ 0 \end{bmatrix}
+ (R_{\phi})^{T}\begin{bmatrix}0 \\ \dot{\theta} \\ 0\end{bmatrix}
+ (R_{\theta}R_{\phi})^{T}\begin{bmatrix}0 \\ 0 \\ \dot{\psi} \end{bmatrix} \\
&= \begin{bmatrix}1 & 0 & -\sin{\theta} \\ 0 & \cos{\phi} & \sin{\phi}\cos{\theta} \\ 0 & -\sin{\phi} & \cos{\theta}\cos{\phi}\end{bmatrix}
\begin{bmatrix} \dot{\phi} \\ \dot{\theta} \\ \dot{\psi}
\end{bmatrix}
\end{align}
$$

$$
\begin{bmatrix} \dot{\phi} \\ \dot{\theta} \\ \dot{\psi}
\end{bmatrix} = \begin{bmatrix}
1 & \sin{\phi}\tan{\theta} & \cos{\phi}\tan{\theta} \\
0 & \cos{\phi} & -\sin{\phi} \\
0 & \frac{\sin{\phi}}{\cos{\theta}} & \frac{\cos{\phi}}{\cos{\theta}}
\end{bmatrix}
\begin{bmatrix} p \\ q \\ r
\end{bmatrix}
$$

In [43]:
droll = ca.SX.sym('droll')
dpitch = ca.SX.sym('dpitch')
dyaw = ca.SX.sym('dyaw')

pqr_mat = ca.vertcat(droll,0,0) + r_roll.T @ ca.vertcat(0,dpitch,0) + (r_pitch @ r_roll).T @ ca.vertcat(0,0,dyaw)

for i in range(0,3):
    print(f"{i}: {pqr_mat[i]}")

0: (droll-(sin(pitch)*dyaw))
1: ((cos(roll)*dpitch)+((cos(pitch)*sin(roll))*dyaw))
2: (((cos(pitch)*cos(roll))*dyaw)-(sin(roll)*dpitch))


In [44]:
pqr_factored = ca.vertcat(
    ca.horzcat(1, 0 , -ca.sin(pitch)),
    ca.horzcat(0, ca.cos(roll), ca.sin(roll)*ca.cos(pitch)),
    ca.horzcat(0, -ca.sin(roll), ca.cos(pitch)*ca.cos(roll)))

pqr_inv = ca.inv(pqr_factored)

for i in range(0,3):
    for j in range(0,3):
        print(pqr_inv[i,j], end=' ')
    print()

@1=cos(roll), @2=(cos(pitch)*cos(roll)), @3=(sin(roll)*cos(pitch)), @4=sin(roll), (((@1*@2)+(@3*@4))/((@1*@2)+(@3*@4))) @1=sin(roll), ((@1*sin(pitch))/((cos(roll)*(cos(pitch)*cos(roll)))+((sin(roll)*cos(pitch))*@1))) @1=cos(roll), ((@1*sin(pitch))/((@1*(cos(pitch)*cos(roll)))+((sin(roll)*cos(pitch))*sin(roll)))) 
00 @1=(cos(pitch)*cos(roll)), (@1/((cos(roll)*@1)+((sin(roll)*cos(pitch))*sin(roll)))) @1=(sin(roll)*cos(pitch)), (-(@1/((cos(roll)*(cos(pitch)*cos(roll)))+(@1*sin(roll))))) 
00 @1=sin(roll), (@1/((cos(roll)*(cos(pitch)*cos(roll)))+((sin(roll)*cos(pitch))*@1))) @1=cos(roll), (@1/((@1*(cos(pitch)*cos(roll)))+((sin(roll)*cos(pitch))*sin(roll)))) 
